In [2]:
import numpy as	np

In [1]:
input_file = '../data/INLP_data/glove.42B.300d.txt'
embeddings_file = input_file[:-4] + '.wv.npy'
vocabulary_file = input_file[:-4] + '.vocab'
words = []
vectors = []

In [3]:
with open(input_file, 'r') as f:
    for line in f:
        fields = line.split()
        if len(fields) == 2:
            print(fields)
            continue
        word = fields[0]
        vector = np.fromiter((float(x) for x in fields[1:]),
                             dtype=np.float)
        words.append(word)
        vectors.append(vector)

matrix = np.array(vectors)
np.save(embeddings_file, matrix)
text = '\n'.join(words)
with open(vocabulary_file, 'w') as f:
    f.write(text)